In [41]:
import pandas as pd
import numpy as np
import os as os
from shutil import rmtree
import cv2 as cv2
import matplotlib.pyplot as plt


Let's set some paramaters:

 * Boarder_Cut: how many pixles to cut off of each edge of the image. Some of the images have noise lines that run through the edges, so we will chip it down using this  
 * Split_Level: THe Test/Train split  
 * Noise_Threshhold: Any pixle below this value will be set to zero  
 * Base_Dir: what is the root director of where the data is stores
 * Train_Dir: where is the training data being stored  
 * Test_Dir: where is the testing Diata being stores  
 * Final_Dir: where is the final test data being stored

In [23]:
Boarder_Cut = 5
Split_Level = 0.85
Noise_Threshhold = 40
Base_Dir = '../Data/'

Train_Dir = Base_Dir + 'Train'
Test_Dir = Base_Dir + 'Test'


IMPORT_LIST = [Base_Dir + 'train_image_data_' + x + '.parquet' for x in ['0','1','2','3']]
IMPORT_VAL_LIST = [Base_Dir + 'test_image_data_' + x + '.parquet' for x in ['0','1','2','3']]
TRAIN_CSV = Base_Dir + 'train.csv'
TEST_CSV = Base_Dir + 'test.csv'


#Df = pd.read_parquet('../Data/train_image_data_3.parquet')
#Df_csv = pd.read_csv(TRAIN_CSV)
#2.48GB in size

In [79]:
def bounding_box(img, Values = False):
    ce = 0
    re = 0
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    
    if Values:
        return rmin, rmax, cmin, cmax
    else:
        if rmin == rmax:
            rmin = 0,
            rmax = img.shape[0]
        if cmin == cmax:
            cmin = 0
            cmax = img.shape[1]
        return(img[rmin:rmax, cmin:cmax])

### Deleting and Making Directories

In [16]:
RR = pd.DataFrame(None)
RR

""


In [88]:
rmtree(Train_Dir)
rmtree(Test_Dir)
os.mkdir(Train_Dir)
os.mkdir(Test_Dir)

Output_csv_File = pd.DataFrame(None)

In [89]:

for IL in IMPORT_LIST:
    print(IL)
    Df = pd.read_parquet(IL)
    

    # Extracting the raw values from the Data frame
    Df_names = pd.DataFrame({'image_id':Df['image_id'].copy(),
                            'Train':0,
                           'Image_Dir': ''})
    
    Df_np = 255 - Df.iloc[:,1:].values.reshape(-1, 137,236).astype(np.uint8).copy()
    
    Df = None

    #cropping size of the image
    Df_np = Df_np[:,Boarder_Cut:-Boarder_Cut,Boarder_Cut:-Boarder_Cut]

    #Reducing noise
    Df_np[Df_np < Noise_Threshhold] = 0

    for Idx in range(Df_np.shape[0]):
        Img = bounding_box(Df_np[Idx,:,:])
        if np.random.uniform() <= Split_Level:
            Save_Dir = Train_Dir + '/' + Df_name.iloc[Idx,0] + '.jpg'
 
        else:    
            Save_Dir = Test_Dir + '/' + Df_name.iloc[Idx,0] + '.jpg'
            Df_names.iloc[Idx,1] = 1
        
        cv2.imwrite(Save_Dir, cv2.resize(Img, (160,90)))
        Df_names.iloc[Idx,2] = Save_Dir
    if Output_csv_File.shape[0] == 0:
        Output_csv_File = Df_names.copy()
    else:
        Output_csv_File = Output_csv_File.append(Df_names, ignore_index = True)
    

../Data/train_image_data_0.parquet
../Data/train_image_data_1.parquet
../Data/train_image_data_2.parquet
../Data/train_image_data_3.parquet


In [94]:
pd.merge(left = pd.read_csv(TRAIN_CSV), right = Output_csv_File, how = 'inner', on = 'image_id').to_csv(Base_Dir + 'train_extra.csv', index = False)

# Now time to format the final Test Group